In [1]:
import pandas as pd
import os

In [2]:
os.chdir("/home/573/rh2942/WASSA-2023-EMP") # changing dir for evaluation file
# print(os.getcwd())

# Prediction

In [3]:
from main_methods import final_prediction

In [4]:
task='personality_conscientiousness'
final_prediction(task=task, lr=5.98e-05, batch=7, seed=30)
predictions_cons = pd.read_csv("./prediction/predictions_" + task + ".tsv", sep='\t', header=None)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Launching training on 4 GPUs.
personality_conscientiousness prediction


Map:   0%|          | 0/1558 [00:00<?, ? examples/s]

Starting epoch 1


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch 0: average loss = 7.293601130268404
Starting epoch 2
Epoch 1: average loss = 0.6038105689055685
Starting epoch 3
Epoch 2: average loss = 0.10767869501640755
Starting epoch 4
Epoch 3: average loss = 0.024517840506242856
Starting epoch 5
Epoch 4: average loss = 0.010254248590042283
Starting epoch 6
Epoch 5: average loss = 0.004560026430616355
Starting epoch 7
Epoch 6: average loss = 0.0048212148753269245
Starting epoch 8
Epoch 7: average loss = 0.004037013725402565
Starting epoch 9
Epoch 8: average loss = 0.0027284501026900087
Starting epoch 10
Epoch 9: average loss = 0.0029975049378013602
Starting epoch 11
Epoch 10: average loss = 0.001400574311836473
Starting epoch 12
Epoch 11: average loss = 0.001029262736275476
Starting epoch 13
Epoch 12: average loss = 0.0007919083475696555
Starting epoch 14
Epoch 13: average loss = 0.0008033025987970177
Starting epoch 15
Epoch 14: average loss = 0.0008267126531791291
Starting epoch 16
Epoch 15: average loss = 0.0005572999947876919
Starting ep

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

In [5]:
task='personality_openess'
final_prediction(task=task, lr=1.80e-05, batch=2, seed=81)
predictions_open = pd.read_csv("./prediction/predictions_" + task + ".tsv", sep='\t', header=None)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Launching training on 4 GPUs.
personality_openess prediction


Map:   0%|          | 0/1558 [00:00<?, ? examples/s]

Starting epoch 1


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch 0: average loss = 4.159229168821222
Starting epoch 2
Epoch 1: average loss = 1.245588234654413
Starting epoch 3
Epoch 2: average loss = 0.17156630597127542
Starting epoch 4
Epoch 3: average loss = 0.05255101512612339
Starting epoch 5
Epoch 4: average loss = 0.01980109081305739
Starting epoch 6
Epoch 5: average loss = 0.007789957879028629
Starting epoch 7
Epoch 6: average loss = 0.004239998647804336
Starting epoch 8
Epoch 7: average loss = 0.003389917279771628
Starting epoch 9
Epoch 8: average loss = 0.003829020760969844
Starting epoch 10
Epoch 9: average loss = 0.002792892811862074
Starting epoch 11
Epoch 10: average loss = 0.002789233651333817
Starting epoch 12
Epoch 11: average loss = 0.00306693250709148
Starting epoch 13
Epoch 12: average loss = 0.0029392545741687244
Starting epoch 14
Epoch 13: average loss = 0.0037579066612707743
Starting epoch 15
Epoch 14: average loss = 0.0031968474365804958
Starting epoch 16
Epoch 15: average loss = 0.004276277466187121
Starting epoch 17
E

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

In [6]:
task='personality_extraversion'
final_prediction(task=task, lr=1.61e-05, batch=7, seed=34)
predictions_extr = pd.read_csv("./prediction/predictions_" + task + ".tsv", sep='\t', header=None)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Launching training on 4 GPUs.
personality_extraversion prediction


Map:   0%|          | 0/1558 [00:00<?, ? examples/s]

Starting epoch 1


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch 0: average loss = 7.641037677015577
Starting epoch 2
Epoch 1: average loss = 3.27992731465825
Starting epoch 3
Epoch 2: average loss = 0.4582739544899336
Starting epoch 4
Epoch 3: average loss = 0.14296371388315623
Starting epoch 5
Epoch 4: average loss = 0.05382057483906725
Starting epoch 6
Epoch 5: average loss = 0.03665799649232732
Starting epoch 7
Epoch 6: average loss = 0.02221645860949398
Starting epoch 8
Epoch 7: average loss = 0.018610271860130263
Starting epoch 9
Epoch 8: average loss = 0.012040372540858308
Starting epoch 10
Epoch 9: average loss = 0.01043257637606335
Starting epoch 11
Epoch 10: average loss = 0.009424931184704681
Starting epoch 12
Epoch 11: average loss = 0.010612345104163978
Starting epoch 13
Epoch 12: average loss = 0.005070136824672643
Starting epoch 14
Epoch 13: average loss = 0.003148098378525382
Starting epoch 15
Epoch 14: average loss = 0.0034210989350477966
Starting epoch 16
Epoch 15: average loss = 0.002021100253322012
Starting epoch 17
Epoch 1

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

In [7]:
task='personality_agreeableness'
final_prediction(task=task, lr=5.15e-05, batch=6, seed=65)
predictions_agre = pd.read_csv("./prediction/predictions_" + task + ".tsv", sep='\t', header=None)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Launching training on 4 GPUs.
personality_agreeableness prediction


Map:   0%|          | 0/1558 [00:00<?, ? examples/s]

Starting epoch 1


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch 0: average loss = 7.831280968739436
Starting epoch 2
Epoch 1: average loss = 1.0885334826432742
Starting epoch 3
Epoch 2: average loss = 0.3914587867374604
Starting epoch 4
Epoch 3: average loss = 0.12260530853930574
Starting epoch 5
Epoch 4: average loss = 0.05401761461085138
Starting epoch 6
Epoch 5: average loss = 0.016522223165688608
Starting epoch 7
Epoch 6: average loss = 0.0077623423900849255
Starting epoch 8
Epoch 7: average loss = 0.00934198091720016
Starting epoch 9
Epoch 8: average loss = 0.003378495245348089
Starting epoch 10
Epoch 9: average loss = 0.003064796048923969
Starting epoch 11
Epoch 10: average loss = 0.0025786486534903254
Starting epoch 12
Epoch 11: average loss = 0.0012174926035536024
Starting epoch 13
Epoch 12: average loss = 0.000811711602504007
Starting epoch 14
Epoch 13: average loss = 0.0011217648765663257
Starting epoch 15
Epoch 14: average loss = 0.0010575346865852435
Starting epoch 16
Epoch 15: average loss = 0.0008316891135249395
Starting epoch 1

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

In [8]:
task='personality_stability'
final_prediction(task=task, lr=5.36e-05, batch=5, seed=13)
predictions_stab = pd.read_csv("./prediction/predictions_" + task + ".tsv", sep='\t', header=None)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Launching training on 4 GPUs.
personality_stability prediction


Map:   0%|          | 0/1558 [00:00<?, ? examples/s]

Starting epoch 1


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch 0: average loss = 5.090471291580261
Starting epoch 2
Epoch 1: average loss = 0.6628222379308099
Starting epoch 3
Epoch 2: average loss = 0.17261609677464151
Starting epoch 4
Epoch 3: average loss = 0.053915723122489184
Starting epoch 5
Epoch 4: average loss = 0.023547655943100557
Starting epoch 6
Epoch 5: average loss = 0.007961300796476932
Starting epoch 7
Epoch 6: average loss = 0.005706934993026945
Starting epoch 8
Epoch 7: average loss = 0.0029397536019155853
Starting epoch 9
Epoch 8: average loss = 0.001651788706229313
Starting epoch 10
Epoch 9: average loss = 0.0037067702438574857
Starting epoch 11
Epoch 10: average loss = 0.002736162735508958
Starting epoch 12
Epoch 11: average loss = 0.0014193213516055271
Starting epoch 13
Epoch 12: average loss = 0.0016011555664403997
Starting epoch 14
Epoch 13: average loss = 0.001589390338220908
Starting epoch 15
Epoch 14: average loss = 0.004762043028886882
Starting epoch 16
Epoch 15: average loss = 0.0036264924669805435
Starting epoc

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

In [9]:
predictions_PER = pd.concat([predictions_cons, predictions_open, predictions_extr, predictions_agre, predictions_stab], axis=1)

predictions_PER.to_csv("./prediction/predictions_PER.tsv", sep='\t', header=False, index=False)

In [10]:
from evaluation import pearsonr, calculate_pearson

In [11]:
# # Just checking the dev set performance
gold_dev = pd.read_csv('./dataset/dev/goldstandard_dev.tsv', sep='\t', header=None) # no header
predictions_PER= pd.read_csv("./prediction/predictions_PER.tsv", sep='\t', header=None)

print(f"cons: {pearsonr(gold_dev.loc[:,3].tolist(), predictions_PER.loc[:,0].tolist())}")
print(f"open: {pearsonr(gold_dev.loc[:,4].tolist(), predictions_PER.loc[:,1].tolist())}")
print(f"extr: {pearsonr(gold_dev.loc[:,5].tolist(), predictions_PER.loc[:,2].tolist())}")
print(f"agre: {pearsonr(gold_dev.loc[:,6].tolist(), predictions_PER.loc[:,3].tolist())}")
print(f"stab: {pearsonr(gold_dev.loc[:,7].tolist(), predictions_PER.loc[:,4].tolist())}")

cons: 0.5956
open: 0.6585
extr: 0.6655
agre: 0.6926
stab: 0.5429


# Hyperparam tuning

In [ ]:
from main_methods import load_tokenised_data

In [ ]:
import torch
import transformers as trf
import numpy as np
import optuna
import plotly
from evaluation import pearsonr, calculate_pearson

In [ ]:
# task = "personality_conscientiousness"
# task = 'personality_openess'
# task = 'personality_extraversion'
# task = 'personality_agreeableness'
task = 'personality_stability'

NUM_EPOCH = 35

# validation by dev set only
train_filename = "train_train_paraphrased.csv"
dev_filename = "dev_summarised.csv"

# Chosen features
feature_1 = 'demographic_essay'
feature_2 = 'article'

checkpoint = "bert-base-uncased"

tokeniser = trf.AutoTokenizer.from_pretrained(checkpoint)
    
#padding="longest" can be deferred to do dynamic padding
def tokenise(sentence):
    return tokeniser(sentence[feature_1], sentence[feature_2], truncation=True)

# data collator due to variable max token length per batch size
data_collator = trf.DataCollatorWithPadding(tokenizer = tokeniser)

In [ ]:
def objective(trial):    
    # Tuning hyperparams:
    LEARNING_RATE = trial.suggest_float("LEARNING_RATE", 1e-05, 1e-04, log=True)
    BATCH_SIZE = trial.suggest_int("BATCH_SIZE", 2, 8)
    SEED = trial.suggest_int("SEED", 1, 100)
    # checkpoint = trial.suggest_categorical("checkpoint", ("bert-base-uncased", "albert-base-v2"))
    
    torch.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    np.random.seed(SEED)
    
    model = trf.AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=1)

    # device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    device = "cuda:1"
    model.to(device)
    
    opt = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    
    ## Split train-test from thw whole train-dev dataset
    # train_dev = load_tokenised_data(filename=os.path.join("./processed_data", train_filename), tokenise_fn=tokenise, train_test="train")
    # train_portion = int(len(train_dev) * 0.8)
    # validation_portion = len(train_dev) - train_portion
    # train_subset, val_subset = torch.utils.data.random_split(train_dev, [train_portion, validation_portion])
    
    # Training by train set only
    train_subset = load_tokenised_data(filename=os.path.join("./processed_data", train_filename), task=task, tokenise_fn=tokenise, train_test="train")
    
    train_dataloader = torch.utils.data.DataLoader(
        train_subset, shuffle=True, batch_size=BATCH_SIZE, collate_fn=data_collator
    ) 
    
    training_steps = NUM_EPOCH * len(train_dataloader)
   
    lr_scheduler = trf.get_scheduler(
        "linear",
        optimizer=opt,
        num_warmup_steps=0,
        num_training_steps=training_steps
    )
    
    model.train()
    for epoch in range(NUM_EPOCH):
        for batch in train_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss

            loss.backward()    
            opt.step()
            lr_scheduler.step()
            opt.zero_grad()

        # Evaluation   
        val_subset = load_tokenised_data(filename=os.path.join("./processed_data", dev_filename), task=task, tokenise_fn=tokenise, train_test="train") #treain_test="train" ensures output labesl are also passed
        validation_dataloader = torch.utils.data.DataLoader(
            val_subset, shuffle=False, batch_size=BATCH_SIZE, collate_fn=data_collator
        )
        model.eval()
        y_true =[]
        y_pred = []

        for batch in validation_dataloader:
            with torch.no_grad():
                batch = {k: v.to(device) for k, v in batch.items()}
                outputs = model(**batch)

            y_true.extend(batch['labels'].tolist())
            batch_pred = [item for sublist in outputs.logits.tolist() for item in sublist]  #convert 2D list to 1D
            y_pred.extend(batch_pred)
        
        pearson_r = pearsonr(y_true, y_pred)
            
        trial.report(pearson_r, epoch)
            
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    
    return pearson_r

In [ ]:
study = optuna.create_study(
    sampler=optuna.samplers.TPESampler(seed=28),
    pruner=optuna.pruners.MedianPruner(),
    direction="maximize"
)
study.optimize(objective, n_trials=50, show_progress_bar=True)

trial_results = study.trials_dataframe() #trial results as a dataframe
trial_results.to_csv("trial_results_" + task + ".csv")

print(f"Best Pearson r: {study.best_value}")
print(f"Best parameter: {study.best_params}")

In [ ]:
fig_1 = optuna.visualization.plot_slice(study)
fig_1.show()
fig_1.write_image("./prediction/" + task + "-param-plots.pdf")

In [ ]:
fig_2 = optuna.visualization.plot_param_importances(study)
fig_2.show()
fig_2.write_image("./prediction/" + task + "-param-importance.pdf")

## Manual

In [ ]:
import csv
from sklearn.model_selection import KFold
from evaluation import pearsonr, calculate_pearson

In [ ]:
def train_test_kfold(model):
    accelerator = Accelerator()
    
    opt = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

    training_steps = NUM_EPOCH * len(trainloader)
    lr_scheduler = trf.get_scheduler(
        "linear",
        optimizer=opt,
        num_warmup_steps=0,
        num_training_steps=training_steps
    )
  
    trainloader_acclerate, model, opt = accelerator.prepare(
        trainloader, model, opt    
    )  
    
    for epoch in range(0, NUM_EPOCH):

        # Print epoch
        accelerator.print(f'Starting epoch {epoch+1}')
        
        epoch_loss = 0
        num_batches = 0

        # Iterate over the DataLoader for training data
        for batch in trainloader_acclerate:
            # Perform forward pass
            outputs = model(**batch)
            
            loss = outputs.loss
#             loss = loss_function(outputs, targets)

            accelerator.backward(loss)
        
            opt.step()
            lr_scheduler.step()
            
            opt.zero_grad()
            
            epoch_loss += loss.item()
            num_batches += 1

        # Process is complete.
        avg_epoch_loss = epoch_loss / num_batches
        accelerator.print(f"Epoch {epoch}: average loss = {avg_epoch_loss}")
    
    
    # Evaluation
        
    model.eval()

    y_true =[]
    y_pred = []

    for batch in testloader:
        with torch.no_grad():
            outputs = model(**batch)

        y_true.extend(batch['labels'].tolist())

        batch_pred = [item for sublist in outputs.logits.tolist() for item in sublist]  #convert 2D list to 1D
        y_pred.extend(batch_pred)
        
    pearson_r = pearsonr(y_true, y_pred)
    
    accelerator.print('\n' + checkpoint + ' & ' + str(LEARNING_RATE) + ' & ' + str(BATCH_SIZE) + ' & ' + feature_1 + '-' + feature_2 + ' & ' + str(pearson_r) + ' fold-' + str(fold) + '\n')

In [ ]:
NUM_EPOCH = 10
BATCH_SIZE = 8
LEARNING_RATE = 5e-5

task = "empathy"

train_dev_filename = "preprocessed_train_dev.csv"

#Chosen features
feature_1 = 'demographic_essay'
feature_2 = 'article'

# feature_1 = 'essay'
# feature_2 = 'demographic'

# feature_1 = 'essay'
# feature_2 ='article'
    
# checkpoint = "bert-base-uncased"
# checkpoint = "distilbert-base-uncased"

tokeniser = trf.AutoTokenizer.from_pretrained(checkpoint)

#padding="longest" can be deferred to do dynamic padding
def tokenise(sentence):
    return tokeniser(sentence[feature_1], sentence[feature_2], truncation=True)
    
# data collator due to variable max token length per batch size
data_collator = trf.DataCollatorWithPadding(tokenizer = tokeniser)

train_dev = load_tokenised_data(filename=os.path.join("./processed_data", train_dev_filename), tokeniser=tokeniser, train_test="train")

K_FOLD = 5

# Set fixed random number seed
torch.manual_seed(42)

kfold = KFold(n_splits=K_FOLD, shuffle=True)

print('--------------------------------')

# K-fold Cross Validation model evaluation
for fold, (train_idx, test_idx) in enumerate(kfold.split(train_dev)):

    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_idx)

    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
        train_dev,
        batch_size=BATCH_SIZE,
        collate_fn=data_collator,
        sampler=train_subsampler
    )
    testloader = torch.utils.data.DataLoader(
        train_dev,
        batch_size=BATCH_SIZE,
        collate_fn=data_collator,
        sampler=test_subsampler
    )

    # Init the neural network
    model = trf.AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=1)

    notebook_launcher(train_test_kfold, (model,), num_processes=torch.cuda.device_count())